# Equity Analysis at the TAZ Level
This Jupyter notebook takes AequilibraE runs (with and without resilience investment) and outputs an HTML file that reports changes in metrics by equity category. 

The default assumption is that the user will run the equity overlay analysis (`run_equity_overlay.bat` file in C:\GitHub\RDR\helper_tools\equity_analysis) as a first step, and then use the output from that as an input to this TAZ metrics analysis. However, the user may also directly provide data in a CSV file assigning an equity variable value to each TAZ from another source, rather than running the equity overlay analysis. If providing other data, the equity data must be numeric. Consult the RDR User Guide in C:\GitHub\RDR\documentation for more information on how to use and understand this tool.

The purpose is to help the user examine and understand differential impacts of a resilience investment intended to mitigate effects of a disruption, comparing different equity categories of interest. The analysis displays variables to help illuminate the following questions from various angles.

Questions driving this analysis include:
- What is the baseline magnitude of trips, minutes per trip, and miles per trip for each equity category?
- How relevant is the disruption for each equity category?
- What is the projected impact of the resilience investment overall and for each equity category, i.e., are the benefits equitably distributed?

The `equity_metrics.config` configuration file allows the user to specify the following:
- `path_to_RDR_config_file` – This should identify the location of the configuration file pertinent to the existing RDR Metamodel run and corresponding AequilibraE runs that will be used for this equity analysis. The analysis will use this configuration file to identify where to access the OMX files from those runs.
- `resil` - Resilience project.
- `hazard`, `recovery`, `socio`, `proj_group`, `elasticity`, `run_type` - Aequilibrae scenario dimensions.
- *Note*: As described above, the default assumption is that the user will use the equity overlay analysis first, and then use the output from that as an input to this TAZ metrics analysis. If the user will instead directly provide the equity data then the user should update the `equity_metrics.config` file (or renamed config file referenced in the run_TAZ_metrics.bat file, if applicable) to specify the name of the user-provided file in `output_name` (without the CSV file extension). The equity data must be numeric.

## Check output directory for CSV file outputs with underlying data results. 
This is in the same location as this HTML file, and is the directory specified in the `equity_analysis_dir` parameter in the equity metrics config file.

## Scroll down in this HTML file for charts and statistical analysis.

In [1]:
# Import statements
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import chisquare

import equity_config_reader

sys.path.insert(0, os.path.abspath('../../metamodel_py'))
import rdr_setup

In [3]:
# The equity configuration filepath is now passed into this notebook from its parent, TAZ_metrics.py, via the temporary text file,
# assuming this notebook and TAZ_metrics.py are both in the same folder.
# To run the notebook in isolation, rather than by executing the run_TAZ_metrics.bat file, comment out the below three lines and 
# uncomment the subsequent two lines.
with open('temp.txt', 'r') as f:
    config_filepath = f.read()
equity_cfg = equity_config_reader.read_equity_config_file(config_filepath)

#config_filepath = "C:\GitHub\RDR\helper_tools\equity_analysis\equity_metrics.config"
#equity_cfg = equity_config_reader.read_equity_config_file(config_filepath)

rdr_cfg_path = equity_cfg['path_to_RDR_config_file']

# Directory of equity helper tool files
equity_dir = equity_cfg['equity_analysis_dir']

cfg = rdr_setup.read_config_file(rdr_cfg_path)

RDR_run_id = cfg['run_id']

# Name of equity variable
category_name = equity_cfg['equity_feature']

# Name of CSV file with equity category value for each TAZ (either output from run_equity_overlay.bat OR user-provided)
category_filename = equity_cfg['output_name']

# P-value for use in statistical tests
pval = float(equity_cfg['pval'])

# Look to see if the equity overlay data exists
if not os.path.exists(os.path.join(equity_dir, category_filename + '.csv')):
    print('{}.csv not found in {}. Please run the equity_overlay first or generate your own file and specify the filename for it as output_name in the equity_metrics.config file'.format(category_filename, equity_dir))

In [4]:
# Utility method for reading OMX files
def readOMX(filename, selectedMatrix, debug_mode):
    f = omx.open_file(filename)
    matrix_size = f.shape()
    if debug_mode:
        print('Shape: ', f.shape())
        print('Number of tables: ', len(f))
        print('Table names: ', f.list_matrices())
        print('Attributes: ', f.list_all_attributes())
    omx_df = f[selectedMatrix]
    if debug_mode:
        print('Sum of matrix elements: ', '{:.9}'.format(np.sum(omx_df)))
        print('Percentiles: ', np.percentile(omx_df, (1, 10, 30, 50, 70, 90, 99)))
        print('Maximum: ', np.amax(omx_df))
    return omx_df, matrix_size, f

In [5]:
# Define inputs for comparison - these come from equity_cfg, the equity_metrics.config file
resil = equity_cfg['resil']
baseline = equity_cfg['baseline']
hazard = equity_cfg['hazard']
recovery = equity_cfg['recovery']
socio = equity_cfg['socio']
projgroup = equity_cfg['projgroup']
elasticity = equity_cfg['elasticity']
elasname = str(int(10 * -elasticity))
run_type = equity_cfg['run_type']
largeval = float(equity_cfg['largeval'])

hours_name = 'free_flow_time'
miles_name = 'distance'

In [6]:
# Location of the "matrix" OMX files for "base"
matrix_omx_folder_path_base = os.path.join(equity_dir, "aeq_runs", "base", RDR_run_id,
                                           socio + projgroup, "matrix", "matrices")

# Location of the "nocar" OMX files for "base"
nocar_omx_folder_path_base = os.path.join(equity_dir, "aeq_runs", "base", RDR_run_id,
                                          socio + projgroup, "nocar", "matrices")

# Location of the "matrix" OMX files for "disruption with resilience investment"
matrix_omx_folder_path = os.path.join(equity_dir, "aeq_runs", "disrupt", RDR_run_id,
                             socio + projgroup + '_' + resil + '_' + elasname + '_' + hazard + '_' + recovery,
                             "matrix", "matrices")

# Location of the "matrix" OMX files for "disruption WITHOUT resilience investment"
matrix_omx_folder_path_noresil = os.path.join(equity_dir, "aeq_runs", "disrupt", RDR_run_id,
                                     socio + projgroup + '_' + baseline + '_' + elasname + '_' + hazard + '_' + recovery,
                                     "matrix", "matrices")

# Location of the "nocar" OMX files for "base" and for "disruption with resilience investment"
nocar_omx_folder_path = os.path.join(equity_dir, "aeq_runs", "disrupt", RDR_run_id,
                             socio + projgroup + '_' + resil + '_' + elasname + '_' + hazard + '_' + recovery,
                             "nocar", "matrices")

# Location of the "nocar" OMX files for "disruption WITHOUT resilience investment"
nocar_omx_folder_path_noresil = os.path.join(equity_dir, "aeq_runs", "disrupt", RDR_run_id,
                                     socio + projgroup + '_' + baseline + '_' + elasname + '_' + hazard + '_' + recovery,
                                     "nocar", "matrices")

# READING THE TABLES FOR "MATRIX"

# Read the base OMX trip table
matrix_base_matrix_filename = os.path.join(matrix_omx_folder_path_base, 'base_demand_summed.omx')
matrix_base_dem, matrix_base_trips_matrix_size, matrix_base_trip_omx_file = readOMX(matrix_base_matrix_filename, 'matrix', 0)
df_matrix_base_trips = pd.DataFrame(data=matrix_base_dem)

# Read the new OMX trip table in the disruption with resilience case
matrix_newdisruptresil_matrix_filename = os.path.join(matrix_omx_folder_path, 'new_demand_summed.omx')
matrix_newdisruptresil_dem, matrix_resil_trips_matrix_size, matrix_newdisruptresil_trip_omx_file = readOMX(matrix_newdisruptresil_matrix_filename, 'matrix', 0)
df_matrix_resil_trips = pd.DataFrame(data=matrix_newdisruptresil_dem)

# Read the new OMX trip table in the disruption WITHOUT resilience case
matrix_newdisruptNOresil_matrix_filename = os.path.join(matrix_omx_folder_path_noresil, 'new_demand_summed.omx')
matrix_newdisruptNOresil_dem, matrix_noresil_trips_matrix_size, matrix_newdisruptNOresil_trip_omx_file = readOMX(matrix_newdisruptNOresil_matrix_filename, 'matrix', 0)
df_matrix_NOresil_trips = pd.DataFrame(data=matrix_newdisruptNOresil_dem)

# READING THE TABLES FOR "NOCAR," if applicable
if os.path.exists(nocar_omx_folder_path):

    # Read the base OMX trip table
    nocar_base_matrix_filename = os.path.join(nocar_omx_folder_path_base, 'base_demand_summed.omx')
    nocar_base_dem, nocar_base_trips_matrix_size, nocar_base_trip_omx_file = readOMX(nocar_base_matrix_filename, 'nocar', 0)
    df_nocar_base_trips = pd.DataFrame(data=nocar_base_dem)

    # Read the new OMX trip table in the disruption with resilience case
    nocar_newdisruptresil_matrix_filename = os.path.join(nocar_omx_folder_path, 'new_demand_summed.omx')
    nocar_newdisruptresil_dem, nocar_resil_trips_matrix_size, nocar_newdisruptresil_trip_omx_file = readOMX(nocar_newdisruptresil_matrix_filename, 'matrix', 0)
    df_nocar_resil_trips = pd.DataFrame(data=nocar_newdisruptresil_dem)

    # Read the new OMX trip table in the disruption WITHOUT resilience case
    nocar_newdisruptNOresil_matrix_filename = os.path.join(nocar_omx_folder_path_noresil, 'new_demand_summed.omx')
    nocar_newdisruptNOresil_dem, nocar_noresil_trips_matrix_size, nocar_newdisruptNOresil_trip_omx_file = readOMX(nocar_newdisruptNOresil_matrix_filename, 'matrix', 0)
    df_nocar_NOresil_trips = pd.DataFrame(data=nocar_newdisruptNOresil_dem)

In [7]:
# Names of file skims
baseskims_filename = run_type + '_' + socio + projgroup
disruptskims_noresil_filename = run_type + '_disrupt_' + socio + projgroup + '_' + baseline + '_' + elasname + '_' + hazard + '_' + recovery
disruptskims_resil_filename = run_type + '_disrupt_' + socio + projgroup + '_' + resil + '_' + elasname + '_' + hazard + '_' + recovery

# Create filepath strings for "matrix" tables
matrix_baseskims_folder = os.path.join(matrix_omx_folder_path_base, baseskims_filename + '.omx')
matrix_disruptskims_noresil_folder = os.path.join(matrix_omx_folder_path_noresil, disruptskims_noresil_filename + '.omx')
matrix_disruptskims_resil_folder = os.path.join(matrix_omx_folder_path, disruptskims_resil_filename + '.omx')

# Create filepath strings for "nocar" tables
nocar_baseskims_folder = os.path.join(nocar_omx_folder_path_base, baseskims_filename + '.omx')
nocar_disruptskims_noresil_folder = os.path.join(nocar_omx_folder_path_noresil, disruptskims_noresil_filename + '.omx')
nocar_disruptskims_resil_folder = os.path.join(nocar_omx_folder_path, disruptskims_resil_filename + '.omx')

In [8]:
# READING THE BASE SKIMS FOR "MATRIX"
# Read the base skims OMX for "matrix"
matrix_base_hours, matrix_base_hours_matrix_size, base_skims_omx_file = readOMX(matrix_baseskims_folder, hours_name, 0)
df_matrix_base_hours = pd.DataFrame(data=matrix_base_hours)
matrix_base_miles, matrix_base_miles_matrix_size, base_skims_omx_file = readOMX(matrix_baseskims_folder, miles_name, 0)
df_matrix_base_miles = pd.DataFrame(data=matrix_base_miles)

# READING THE BASE SKIMS FOR "NOCAR," if applicable
if os.path.exists(nocar_omx_folder_path):
    # Read the base skims OMX for "nocar"
    nocar_base_hours, nocar_base_hours_matrix_size, base_skims_omx_file = readOMX(nocar_baseskims_folder, hours_name, 0)
    df_nocar_base_hours = pd.DataFrame(data=nocar_base_hours)
    nocar_base_miles, nocar_base_miles_matrix_size, base_skims_omx_file = readOMX(nocar_baseskims_folder, miles_name, 0)
    df_nocar_base_miles = pd.DataFrame(data=nocar_base_miles)

In [9]:
# READING THE DISRUPTION WITH NO RESILIENCE SKIMS FOR "MATRIX"
# Read the disrupt skims OMX - no resilience project
matrix_disrupt_noresil_hours, matrix_size, disrupt_noresil_skims_omx_file = readOMX(matrix_disruptskims_noresil_folder, hours_name, 0)
df_matrix_disrupt_noresil_hours = pd.DataFrame(data=matrix_disrupt_noresil_hours)
matrix_disrupt_noresil_miles, matrix_size, disrupt_noresil_skims_omx_file = readOMX(matrix_disruptskims_noresil_folder, miles_name, 0)
df_matrix_disrupt_noresil_miles = pd.DataFrame(data=matrix_disrupt_noresil_miles)

# READING THE DISRUPTION WITH NO RESILIENCE SKIMS FOR "NOCAR," if applicable
if os.path.exists(nocar_omx_folder_path):
    # Read the disrupt skims OMX - no resilience project
    nocar_disrupt_noresil_hours, matrix_size, disrupt_noresil_skims_omx_file = readOMX(nocar_disruptskims_noresil_folder, hours_name, 0)
    df_nocar_disrupt_noresil_hours = pd.DataFrame(data=nocar_disrupt_noresil_hours)
    nocar_disrupt_noresil_miles, matrix_size, disrupt_noresil_skims_omx_file = readOMX(nocar_disruptskims_noresil_folder, miles_name, 0)
    df_nocar_disrupt_noresil_miles = pd.DataFrame(data=nocar_disrupt_noresil_miles)

In [10]:
# READING THE DISRUPTION WITH RESILIENCE SKIMS FOR "MATRIX"
# Read the disrupt skims OMX - with resilience project
matrix_disrupt_resil_hours, matrix_size, disrupt_resil_skims_omx_file = readOMX(matrix_disruptskims_resil_folder, hours_name, 0)
df_matrix_disrupt_resil_hours = pd.DataFrame(data=matrix_disrupt_resil_hours)
matrix_disrupt_resil_miles, matrix_size, disrupt_resil_skims_omx_file = readOMX(matrix_disruptskims_resil_folder, miles_name, 0)
df_matrix_disrupt_resil_miles = pd.DataFrame(data=matrix_disrupt_resil_miles)

# READING THE DISRUPTION WITH RESILIENCE SKIMS FOR "NOCAR," if applicable
if os.path.exists(nocar_omx_folder_path):
    # Read the disrupt skims OMX - with resilience project
    nocar_disrupt_resil_hours, matrix_size, disrupt_resil_skims_omx_file = readOMX(nocar_disruptskims_resil_folder, hours_name, 0)
    df_nocar_disrupt_resil_hours = pd.DataFrame(data=nocar_disrupt_resil_hours)
    nocar_disrupt_resil_miles, matrix_size, disrupt_resil_skims_omx_file = readOMX(nocar_disruptskims_resil_folder, miles_name, 0)
    df_nocar_disrupt_resil_miles = pd.DataFrame(data=nocar_disrupt_resil_miles)

In [11]:
# Function to create dataframe based on skim results
def makeskimresult_df(hours_df,trips_df,miles_df):
    # Base times and distances by origin TAZ
    # Convert O-D matrix to tall table indexed by origin and destination TAZ
    bool_base_hours = hours_df < largeval
    a = np.repeat(bool_base_hours.columns, len(bool_base_hours.index))
    b = np.tile(bool_base_hours.index, len(bool_base_hours.columns))

    # Sums demand where <largeval
    base_cumtripcount = (trips_df.where(bool_base_hours, other=0))
    base_cumtime = (base_cumtripcount*hours_df)/60
    base_cumdist = (base_cumtripcount*miles_df)
    c1 = base_cumtripcount.values.ravel()
    c2 = base_cumtime.values.ravel()
    c3 = base_cumdist.values.ravel()
    df = pd.DataFrame({'from':a, 'to':b, 'trips':c1, 'hours':c2, 'miles':c3})
    return df

In [12]:
# Make dataframes for "matrix"
matrix_base_df = makeskimresult_df(df_matrix_base_hours,df_matrix_base_trips,df_matrix_base_miles)
matrix_disrupt_noresil_df = makeskimresult_df(df_matrix_disrupt_noresil_hours,df_matrix_NOresil_trips,df_matrix_disrupt_noresil_miles)
matrix_disrupt_resil_df = makeskimresult_df(df_matrix_disrupt_resil_hours,df_matrix_resil_trips,df_matrix_disrupt_resil_miles)

if os.path.exists(nocar_omx_folder_path):
    # Make dataframes for "nocar," if applicable
    nocar_base_df = makeskimresult_df(df_nocar_base_hours,df_nocar_base_trips,df_nocar_base_miles)
    nocar_disrupt_noresil_df = makeskimresult_df(df_nocar_disrupt_noresil_hours,df_nocar_NOresil_trips,df_nocar_disrupt_noresil_miles)
    nocar_disrupt_resil_df = makeskimresult_df(df_nocar_disrupt_resil_hours,df_nocar_resil_trips,df_nocar_disrupt_resil_miles)

if os.path.exists(nocar_omx_folder_path):
    # If "nocar" tables exist, combine "matrix" and "nocar" dataframes for overall depiction of results
    base_df = matrix_base_df.add(nocar_base_df)
    disrupt_noresil_df = matrix_disrupt_noresil_df.add(nocar_disrupt_noresil_df)
    disrupt_resil_df = matrix_disrupt_resil_df.add(nocar_disrupt_resil_df)
else:
    # Otherwise if "nocar" tables do not exist, the overall results are just those from the "matrix" folders
    base_df = matrix_base_df
    disrupt_noresil_df = matrix_disrupt_noresil_df
    disrupt_resil_df = matrix_disrupt_resil_df  

In [13]:
matrix_base_trip_omx_file.close()
matrix_newdisruptresil_trip_omx_file.close()
matrix_newdisruptNOresil_trip_omx_file.close()

if os.path.exists(nocar_omx_folder_path):
    nocar_base_trip_omx_file.close()
    nocar_newdisruptresil_trip_omx_file.close()
    nocar_newdisruptNOresil_trip_omx_file.close()

base_skims_omx_file.close()
disrupt_noresil_skims_omx_file.close()
disrupt_resil_skims_omx_file.close()

In [14]:
# Create data frame of skim results
merged_df = pd.merge(base_df, disrupt_noresil_df, how='inner', on=['from', 'to'], suffixes=("_base", None))
taz_pair_skims = pd.merge(merged_df, disrupt_resil_df, how='inner', on=['from', 'to'], suffixes=("_disrupt_noresil", "_disrupt_resil"))

In [15]:
# Read in equity category label by TAZ
taz_equity = pd.read_csv(os.path.join(equity_dir, category_filename + '.csv'),
                         usecols=['TAZ', category_name],
                         converters={'TAZ': int, category_name: float})

In [16]:
# Join by from TAZ and to TAZ
taz_stats = taz_pair_skims.merge(taz_equity, how='left', left_on='from', right_on='TAZ').merge(taz_equity, how='left', left_on='to', right_on='TAZ', suffixes=('_from', '_to'))

# Replace NaN values with 'external'. These are for nodes which do not exist in the TAZ file, and therefore do not have any equity attributes. 
# They are nodes which are outside the MPO boundaries and are needed for travel demand modeling purposes, but do not have shapes associated with them. 
# They are not omitted because the totals for hours, miles, and trips should be the same at the MPO level as what is reported to users.
taz_stats[['TAZ_from', category_name + '_from','TAZ_to', category_name + '_to']] = taz_stats[['TAZ_from', category_name + '_from','TAZ_to', category_name + '_to']].fillna('external')

# Calculate relative change in trips/hours/miles for each 
taz_stats['trips_delta'] = (taz_stats['trips_disrupt_resil'] - taz_stats['trips_disrupt_noresil'])
taz_stats['hours_delta'] = (taz_stats['hours_disrupt_resil'] - taz_stats['hours_disrupt_noresil'])
taz_stats['miles_delta'] = (taz_stats['miles_disrupt_resil'] - taz_stats['miles_disrupt_noresil'])

In [17]:
# Create three variables to flag whether the disruption is relevant for the TAZ pair (for trips/miles/hours)
taz_stats['trips_disruption_relevant'] = taz_stats['trips_base'] != taz_stats['trips_disrupt_noresil']
taz_stats['hours_disruption_relevant'] = taz_stats['hours_base'] != taz_stats['hours_disrupt_noresil']
taz_stats['miles_disruption_relevant'] = taz_stats['miles_base'] != taz_stats['miles_disrupt_noresil']

In [18]:
# Create a function to produce a summary table given an index of interest (i.e., a grouping of TAZ based on equity category of origin/destination).

# First create helper function for use in the main function
def countnonzeros(x):
    return x.astype(bool).sum(axis=0)

# Now create main function
def createsummary(index):
    # First address TRIPS
    metric = 'trips'
    # Aggregate base metric for all three categories (Variables 1T/1H/1M)
    category_base = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_base':np.sum}, 
                                    fill_value=0)
    category_base = pd.DataFrame(category_base.to_records())
    # Create variables 2aT/2aH/2aM: Percent change from baseline due to disruption (without resilience investment) and merge with the prior into one dataframe
    Q_TwoA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_noresil':np.sum}, 
                                    fill_value=0)
    category_stats = pd.merge(category_base,Q_TwoA,on=None,left_index=True, right_index=True)
    category_stats[metric+'_absolute_change_noresil'] = category_stats[metric+'_disrupt_noresil'] - category_stats[metric+'_base']
    category_stats[metric+'_percent_change_noresil'] = (category_stats[metric+'_absolute_change_noresil']*100)/category_stats[metric+'_base']

    # Create variables 2bT/2bH/2bM: Percent of TAZ with a change in metric due to disruption (without resilience investment)
    Q_TwoB = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disruption_relevant':[countnonzeros,len]}, 
                                    fill_value=0)
    # Flatten multi index hierarchy in column headers
    Q_TwoB = pd.DataFrame(Q_TwoB.to_records())
    Q_TwoB[metric+'_percent_TAZ_relevant'] = (Q_TwoB["('"+metric+"_disruption_relevant', 'countnonzeros')"]*100)/Q_TwoB["('"+metric+"_disruption_relevant', 'len')"]
    # Merge into category_stats dataframe
    category_stats = pd.merge(category_stats,Q_TwoB,on=index)

    # Create variables 3aT/3aH/3aM: Overall impact of resilience investment (i.e., metric in the "resilience" case minus metric in the "no resilience" case)
    Q_ThreeA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_resil':np.sum}, 
                                    fill_value=0)
    category_stats = pd.merge(category_stats,Q_ThreeA,on=index)
    category_stats[metric+"_delta_absolute"] = category_stats[metric+'_disrupt_resil'] - category_stats[metric+'_disrupt_noresil']
    # Create variables 3bT/3bH/3bM: Same as the above set, except divided by the metric in the "no resilience" case and multiplied by 100 to show percent change relative to "no resilience" case
    category_stats[metric+'_delta_relative'] = (category_stats[metric+'_delta_absolute']*100)/category_stats[metric+'_disrupt_noresil']

    # Average difference in metrics due to resilience investment for ALL TAZ 
    AvgDeltaforAll = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_delta':np.mean}, 
                                    fill_value=0)
    AvgDeltaforAll = AvgDeltaforAll.rename(columns = {metric+'_delta':metric+'_mean_delta_for_ALL'})
    if metric+'_mean_delta_for_ALL' in AvgDeltaforAll.columns.to_list():
        category_stats = pd.merge(category_stats,AvgDeltaforAll,on=index)

    # Create variables 3cT/3cH/3cM: Average difference in metric due to resilience investment for all relevant TAZ pairs (i.e., among the subset of TAZ pairs where there was a disruption impact in the "no resilience" case)
    # First filter for relevant TAZ pairs
    relevant_filter = taz_stats[metric+'_disrupt_noresil'] != taz_stats[metric+'_base'] 
    relevant_set = taz_stats[relevant_filter]
    Q_ThreeC = pd.pivot_table(relevant_set, index=index, values=relevant_set.columns.to_list(),
                                    aggfunc={metric+'_delta':np.mean}, 
                                    fill_value=0)
    Q_ThreeC = Q_ThreeC.rename(columns = {metric+'_delta':metric+'_mean_delta_for_relevant_pairs'})
    if metric+'_mean_delta_for_relevant_pairs' in Q_ThreeC.columns.to_list():
        category_stats = pd.merge(category_stats,Q_ThreeC,on=index)

    # Create variables 3dT/3dH/3dM: Average difference in metric due to resilience investment for all TAZ pairs with non-zero delta due to resilience (i.e., among the even smaller subset of TAZ pairs where the "resilience" case was different from the "no resilience" case)
    nonzerodelta_filter = taz_stats[metric+'_disrupt_noresil'] != taz_stats[metric+'_disrupt_resil']
    nonzerodelta_set = taz_stats[nonzerodelta_filter]
    Q_ThreeD = pd.pivot_table(nonzerodelta_set, index=index, values=nonzerodelta_set.columns.to_list(),
                                aggfunc={metric+'_delta':np.mean}, 
                                fill_value=0)
    Q_ThreeD = Q_ThreeD.rename(columns = {metric+'_delta':metric+'_mean_delta_for_pairs_with_non-zero_delta'})
    if metric+'_mean_delta_for_pairs_with_non-zero_delta' in Q_ThreeD.columns.to_list():
        category_stats = pd.merge(category_stats,Q_ThreeD,on=index)


    # HOURS PER TRIP
    metric = 'hours'
    hours_stats = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_base':np.sum}, 
                                    fill_value=0)    
    hours_stats = pd.DataFrame(hours_stats.to_records())
    # Pull in necessary prerequisite values for hours
    Q_TwoA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_noresil':np.sum}, 
                                    fill_value=0)
    hours_stats = pd.merge(hours_stats,Q_TwoA,on=index)

    Q_ThreeA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_resil':np.sum}, 
                                    fill_value=0)
    hours_stats = pd.merge(hours_stats,Q_ThreeA,on=index)

    # Merge into category_stats
    category_stats = pd.merge(category_stats,hours_stats,on=None,left_index=True, right_index=True)
    
    # Create variables 2aT/2aH/2aM: Percent change from baseline "minutes per trip" due to disruption (without resilience investment)
    category_stats['minutespertrip_base'] = (category_stats[metric+'_base']*60)/category_stats['trips_base']
    category_stats['minutespertrip_disrupt_noresil'] = (category_stats[metric+'_disrupt_noresil']*60)/category_stats['trips_disrupt_noresil']
    
    category_stats['minutespertrip_absolute_change_noresil'] = category_stats['minutespertrip_disrupt_noresil'] - category_stats['minutespertrip_base']
    category_stats['minutespertrip_percent_change_noresil'] = (category_stats['minutespertrip_absolute_change_noresil']*100)/category_stats['minutespertrip_base']    
    
    category_stats['minutespertrip_disrupt_resil'] = (category_stats[metric+'_disrupt_resil']*60)/category_stats['trips_disrupt_resil']
    
    # No need to do question 2B (percent TAZ relevant) because we would have already captured this with trips

    # Create variables 3aT/3aH/3aM: Overall impact of resilience investment in minutes per trip (i.e., minutes per trip in the "resilience" case minus minutes per trip in the "no resilience" case)
    category_stats["minutespertrip_delta_absolute"] = category_stats['minutespertrip_disrupt_resil'] - category_stats['minutespertrip_disrupt_noresil']
    
    # Create variables 3bT/3bH/3bM: Same as the above set, except divided by the minutes per trip in the "no resilience" case and multiplied by 100 to show percent change relative to "no resilience" case
    category_stats['minutespertrip_delta_relative'] = (category_stats['minutespertrip_delta_absolute']*100)/category_stats['minutespertrip_disrupt_noresil']

    # MILES PER TRIP
    metric = 'miles'
    miles_stats = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_base':np.sum}, 
                                    fill_value=0)    
    miles_stats = pd.DataFrame(miles_stats.to_records())
    # Pull in necessary prerequisite values for miles
    Q_TwoA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_noresil':np.sum}, 
                                    fill_value=0)
    miles_stats = pd.merge(miles_stats,Q_TwoA,on=index)

    Q_ThreeA = pd.pivot_table(taz_stats, index=index, values=taz_stats.columns.to_list(),
                                    aggfunc={metric+'_disrupt_resil':np.sum}, 
                                    fill_value=0)
    miles_stats = pd.merge(miles_stats,Q_ThreeA,on=index)

    # Merge into category_stats
    category_stats = pd.merge(category_stats,miles_stats,on=None,left_index=True, right_index=True)
    
    # Create variables 2aT/2aH/2aM: Percent change from baseline "miles per trip" due to disruption (without resilience investment)
    category_stats['milespertrip_base'] = (category_stats[metric+'_base'])/category_stats['trips_base']
    category_stats['milespertrip_disrupt_noresil'] = (category_stats[metric+'_disrupt_noresil'])/category_stats['trips_disrupt_noresil']

    category_stats['milespertrip_absolute_change_noresil'] = category_stats['milespertrip_disrupt_noresil'] - category_stats['milespertrip_base']
    category_stats['milespertrip_percent_change_noresil'] = (category_stats['milespertrip_absolute_change_noresil']*100)/category_stats['milespertrip_base']            

    category_stats['milespertrip_disrupt_resil'] = (category_stats[metric+'_disrupt_resil'])/category_stats['trips_disrupt_resil']

    # No need to do question 2B (percent TAZ relevant) because we would have already captured this with trips

    # Create variables 3aT/3aH/3aM: Overall impact of resilience investment in miles per trip (i.e., miles per trip in the "resilience" case minus miles minutes per trip in the "no resilience" case)
    category_stats["milespertrip_delta_absolute"] = category_stats['milespertrip_disrupt_resil'] - category_stats['milespertrip_disrupt_noresil']
    
    # Create variables 3bT/3bH/3bM: Same as the above set, except divided by the miles per trip in the "no resilience" case and multiplied by 100 to show percent change relative to "no resilience" case
    category_stats['milespertrip_delta_relative'] = (category_stats['milespertrip_delta_absolute']*100)/category_stats['milespertrip_disrupt_noresil']
    
    # Subset the columns of interest
    category_stats = category_stats.filter([index,
                                            'trips_base', 

                                            'trips_disrupt_noresil',
                                            'trips_disrupt_resil',
                                            #"('trips_disruption_relevant', 'countnonzeros')"
                                            #"('trips_disruption_relevant', 'len')"
                                            
                                            'trips_absolute_change_noresil',
                                            'trips_percent_change_noresil',
                                            'trips_percent_TAZ_relevant',
                                            'trips_delta_absolute', 
                                            'trips_delta_relative',

                                            'trips_mean_delta_for_ALL',

                                            'trips_mean_delta_for_relevant_pairs',

                                            'trips_mean_delta_for_pairs_with_non-zero_delta',
                                            
                                            'hours_base', 
                                            'hours_disrupt_noresil',
                                            'hours_disrupt_resil',
                                            
                                            'minutespertrip_base', 
                                            'minutespertrip_absolute_change_noresil',
                                            'minutespertrip_percent_change_noresil', 
                                            'minutespertrip_delta_absolute', 
                                            'minutespertrip_delta_relative',
                                            
                                            'miles_base', 
                                            'miles_disrupt_noresil',
                                            'miles_disrupt_resil',
                                                                                        
                                            'milespertrip_base', 
                                            'milespertrip_absolute_change_noresil',
                                            'milespertrip_percent_change_noresil', 
                                            'milespertrip_delta_absolute', 
                                            'milespertrip_delta_relative'],
                                       axis=1)

    # Convert the equity category to string for better rendering in the charts that follow.
    category_stats[index] = category_stats[index].astype(str)
    return category_stats

In [19]:
# Produce summary table for trips, minutes per trip, and miles per trip
summary = createsummary(category_name + '_from')

In [20]:
# Create another function to aggregate and calculate metrics by TAZ of origin or destination - 
# This one is less aggregated than the "create summary" function above. Instead of each row
# being a equity category, each row is a TAZ and trips are assigned either by TAZ of origin or 
# TAZ of destination. The sole purpose of this is to produce .csv outputs for the user that 
# parallel the .csv outputs that the user would get from the continuous version of the notebook.
def aggregate(from_or_to):
    summary = pd.pivot_table(taz_pair_skims, index=from_or_to, values=taz_pair_skims.columns.to_list(),
                                      aggfunc={'trips_base':np.sum,
                                               'trips_disrupt_noresil':np.sum,
                                               'trips_disrupt_resil': np.sum,
                                               'hours_base':np.sum,
                                               'hours_disrupt_noresil':np.sum,
                                               'hours_disrupt_resil': np.sum,
                                               'miles_base':np.sum,
                                               'miles_disrupt_noresil':np.sum,
                                               'miles_disrupt_resil': np.sum,
                                               }, 
                                               fill_value=0)
    summary = pd.DataFrame(summary.to_records())

    # MINUTES PER TRIP calculations
    summary['minutespertrip_base'] = (summary['hours_base']*60)/summary['trips_base']
    summary['minutespertrip_disrupt_noresil'] = (summary['hours_disrupt_noresil']*60)/summary['trips_disrupt_noresil']
    summary['minutespertrip_disrupt_resil'] = (summary['hours_disrupt_resil']*60)/summary['trips_disrupt_resil']
    # MILES PER TRIP calculations
    summary['milespertrip_base'] = (summary['miles_base'])/summary['trips_base']
    summary['milespertrip_disrupt_noresil'] = (summary['miles_disrupt_noresil'])/summary['trips_disrupt_noresil']
    summary['milespertrip_disrupt_resil'] = (summary['miles_disrupt_resil'])/summary['trips_disrupt_resil']    
    # Additional trip calculations
    metric = "trips"
    summary[metric+'_percent_change_noresil'] = ((summary[metric+'_disrupt_noresil'] - summary[metric+'_base'])*100)/summary[metric+'_base']
    summary[metric+"_delta_absolute"] = summary[metric+'_disrupt_resil'] - summary[metric+'_disrupt_noresil']
    summary[metric+'_delta_relative'] = (summary[metric+'_delta_absolute']*100)/summary[metric+'_disrupt_noresil']
    # Additional minutes per trip calculations
    metric = "minutespertrip"
    summary[metric+'_percent_change_noresil'] = ((summary[metric+'_disrupt_noresil'] - summary[metric+'_base'])*100)/summary[metric+'_base']
    summary[metric+"_delta_absolute"] = summary[metric+'_disrupt_resil'] - summary[metric+'_disrupt_noresil']
    summary[metric+'_delta_relative'] = (summary[metric+'_delta_absolute']*100)/summary[metric+'_disrupt_noresil']
    # Additional miles per trip calculations
    metric = "milespertrip"
    summary[metric+'_percent_change_noresil'] = ((summary[metric+'_disrupt_noresil'] - summary[metric+'_base'])*100)/summary[metric+'_base']
    summary[metric+"_delta_absolute"] = summary[metric+'_disrupt_resil'] - summary[metric+'_disrupt_noresil']
    summary[metric+'_delta_relative'] = (summary[metric+'_delta_absolute']*100)/summary[metric+'_disrupt_noresil']    

    # Join by 'from' TAZ or 'to' TAZ (as the case may be)
    summary = summary.merge(taz_equity, how='left', left_on=from_or_to, right_on='TAZ')

    return summary

In [21]:
# First produce a CSV output at the most aggregated level (rows are equity category)
csv_summary_filepath = os.path.join(equity_dir,"MetricsByTAZ_summary_{}_byEquityCategory.csv".format(equity_cfg['run_id']))
# Produce a summary CSV file
summary.to_csv(csv_summary_filepath)

In [22]:
# Next produce two CSV outputs that are a bit less aggregated (rows are TAZ and trips grouped by either origin or destination TAZ)

# By origin TAZ
TAZ_origin_stats = aggregate('from')
origin_csv_summary_filepath = os.path.join(equity_dir,"MetricsByTAZ_summary_{}_byTAZofOrigin.csv".format(equity_cfg['run_id']))
# Produce a summary CSV file
TAZ_origin_stats.to_csv(origin_csv_summary_filepath)

# By destination TAZ
TAZ_destination_stats = aggregate('to')
destination_csv_summary_filepath = os.path.join(equity_dir,"MetricsByTAZ_summary_{}_byTAZofDestination.csv".format(equity_cfg['run_id']))
# Produce a summary CSV file
TAZ_destination_stats.to_csv(destination_csv_summary_filepath)

In [23]:
# Create prerequisite dictionaries and list to use the functions that create charts
ylabel_dict = {"percent_change_noresil":"Percent Change in ",
"delta_absolute":"Change in ",
"delta_relative":"Percent Change in ",
"base":"",
"percent_TAZ_relevant":"Percent of TAZ"}

title_dict = {"percent_change_noresil":"Percent Change from Baseline Due to Disruption (without Resilience) (i.e., Computing Difference in Metric Compared to Base Value, Then Dividing by Base Value)",
"delta_absolute":"Overall Impact of Resilience Investment as Compared to 'No Resilience' Case, for All TAZ (i.e., Metric in 'Resilience' Case Minus Metric in 'No Resilience' Case)",
"delta_relative":"Relative Impact of Resilience Investment as Compared to 'No Resilience' Case, for All TAZ (i.e., Overall Impact Divided by Value of Metric in 'No Resilience' Case)",
"base":"Baseline Magnitude of Metrics for Each Equity Indicator Category, Absent Disruption",
"percent_TAZ_relevant":"Percent of TAZ with Potential Impacts from Disruption"}

color_dict = {"percent_change_noresil":'#024a70',
"delta_absolute":"#990000",
"delta_relative":'#833C0C',
"base":"#548235",
"percent_TAZ_relevant":"#7030A0"}

metrics_list = ["base",
"percent_change_noresil",
"delta_absolute",
"delta_relative",
"percent_TAZ_relevant"]

y_hoverformat_dict = {"percent_change_noresil":"%{y:.3}%",
"delta_absolute":"%{y:,.5}",
"delta_relative":"%{y:.3}%",
"base":"%{y:,.7}",
"percent_TAZ_relevant":"%{y:.3}%"}

label_dict = {"percent_change_noresil":'{:.2f}%',
"delta_absolute":'{:,.2f}',
"delta_relative":'{:.2f}%',
"base":'{:,.2f}',
"percent_TAZ_relevant":'{:.2f}%'}

In [24]:
# Create function to generate bar charts for each question of interest
def makebarcharts(variabletype):
    if "trips_"+variabletype in summary.columns.to_list():
        if (summary['trips_'+variabletype] == 0).all() and (summary['minutespertrip_'+variabletype] == 0).all() and (summary['milespertrip_'+variabletype] == 0).all():
            print("The '___{}' variable was zero for all categories for trips, minutes per trip, and miles per trip, so no chart was produced.".format(variabletype))
        else:
            fig = make_subplots(rows=1, cols=3)
            fig.add_trace(go.Bar(
                y= summary["trips_"+variabletype],
                x= summary[category_name + '_from'],
                marker=dict(color=color_dict[variabletype]),
                hovertemplate=
                "Equity Category of Origin TAZ ("+category_name+")"+": %{x:.1f}<br>" +
                ylabel_dict[variabletype]+"Trips: "+y_hoverformat_dict[variabletype]+"<br>" +
                "<extra></extra>",
                text= summary["trips_"+variabletype].map(label_dict[variabletype].format)),
                row=1, col=1)
            fig.add_trace(go.Bar(
                y= summary["minutespertrip_"+variabletype],
                x= summary[category_name + '_from'],
                marker=dict(color=color_dict[variabletype]),
                hovertemplate=
                "Equity Category of Origin TAZ ("+category_name+")"+": %{x:.1f}<br>" +
                ylabel_dict[variabletype]+"Minutes per Trip: "+y_hoverformat_dict[variabletype]+"<br>" +
                "<extra></extra>",
                text= summary["minutespertrip_"+variabletype].map(label_dict[variabletype].format)),
                row=1, col=2)
            fig.add_trace(go.Bar(
                y= summary["milespertrip_"+variabletype],
                x= summary[category_name + '_from'],
                marker=dict(color=color_dict[variabletype]),
                hovertemplate=
                "Equity Category of Origin TAZ ("+category_name+")"+": %{x:.1f}<br>" +
                ylabel_dict[variabletype]+"Miles per Trip: "+y_hoverformat_dict[variabletype]+"<br>" +
                "<extra></extra>",
                text= summary["milespertrip_"+variabletype].map(label_dict[variabletype].format)),
                row=1, col=3)

            # edit axis labels
            fig['layout']['xaxis']['title']="Equity Category of Origin TAZ ("+category_name+")"
            fig['layout']['xaxis2']['title']="Equity Category of Origin TAZ ("+category_name+")"
            fig['layout']['xaxis3']['title']="Equity Category of Origin TAZ ("+category_name+")"  
            fig['layout']['yaxis']['title']=ylabel_dict[variabletype]+"Trips"
            fig['layout']['yaxis2']['title']=ylabel_dict[variabletype]+"Minutes per Trip"
            fig['layout']['yaxis3']['title']=ylabel_dict[variabletype]+"Miles per Trip"

            # Update title and height
            fig.update_layout(title_text=title_dict[variabletype], height=700, showlegend=False)

            fig.show()

## Questions and corresponding variables
### Question 1: What is the baseline magnitude of the metric for each category?
- Variables: Overall metric for each category absent disruption
    - `trips_base`
    - `minutespertrip_base`
    - `milespertrip_base`

In [25]:
makebarcharts("base")

    
### Question 2: How relevant is the disruption for each category?
##### Question 2A: What was the relative change in the travel metrics due to disruption without resilience?
- Variables: Percent change from baseline metric due to disruption (without resilience investment)
    - `trips_percent_change_noresil`
    - `minutespertrip_percent_change_noresil`
    - `milespertrip_percent_change_noresil` 

In [26]:
makebarcharts("percent_change_noresil")

##### Question 2B: What percent of TAZ were impacted by the disruption without resilience?
- Variable 2bT: Percent of TAZ with a change in trips due to disruption (without resilience investment)
    - `trips_percent_TAZ_relevant`

In [27]:
# 'Percent of TAZ with Potential Impacts from Disruption' is different from the other variable/question types,
# so use the below code to generate a chart instead of using the previously defined functions.
variabletype = "percent_TAZ_relevant"

if "trips_"+variabletype in summary.columns.to_list():
    if (summary['trips_'+variabletype] == 0).all() and (summary['minutespertrip_'+variabletype] == 0).all() and (summary['milespertrip_'+variabletype] == 0).all():
        print("The '___{}' variable was zero for all categories for trips, minutes per trip, and miles per trip, so no chart was produced.".format(variabletype))
    else:
        '''fig = px.bar(summary, x=category_name + '_from', y="trips_"+variabletype,
                     labels={category_name + '_from': "Equity Indicator Groups Based on Origin TAZ",
                             "trips_"+variabletype: "Percent of TAZ"},
                             title=title)'''
        fig = go.Figure(go.Bar(
            y= summary["trips_"+variabletype],
            x= summary[category_name + '_from'],
            marker=dict(color=color_dict[variabletype]),
            hovertemplate=
            category_name + ' of Origin TAZ'+": %{x:.1f}<br>" +
            ylabel_dict[variabletype]+": "+
            y_hoverformat_dict[variabletype]+"<br>" +
                "<extra></extra>",
            text= summary["trips_"+variabletype].map(label_dict[variabletype].format)))
        # edit axis labels
        fig['layout']['xaxis']['title']=category_name+" of Origin TAZ"   
        fig['layout']['yaxis']['title']=ylabel_dict[variabletype]

        # Update title and height
        fig.update_layout(title_text=title_dict[variabletype], height=700, showlegend=False)
        
        fig.show()

### Question 3: What is the projected impact of the resilience investment for this category?
##### Question 3A: What was the absolute impact (change in metric) across all TAZ?
- Variables: Overall impact of resilience investment on metrics (i.e., magnitude in the "resilience" case minus magnitude in the "no resilience" case)
    - `trips_delta_absolute`
    - `minutespertrip_delta_absolute`
    - `milespertrip_delta_absolute`

In [28]:
makebarcharts("delta_absolute")

##### Question 3B: What was the relative impact (change in metric expressed as a percentage of the "no resilience" magnitude) across all TAZ?
- Variables: Same as the above set, except divided by the magnitude in the "no resilience" case and multiplied by 100 to show percent change relative to "no resilience" case
    - `trips_delta_relative`
    - `minutespertrip_delta_relative`
    - `milespertrip_delta_relative`

In [29]:
makebarcharts("delta_relative")

# Statistically Significant Differences between Groups
## Context
The charts above show differences in various metrics across the equity groups (where each "group" is formed based on the equity indicator value of the origin TAZ). Ultimately, to understand whether the benefits of the resilience investment are equitably distributed across these groups, one key question is whether the proportions of impacted trips in the "resilience" case deviate significantly from the expected proportions, where our expectation is based on the corresponding proportions in the "no resilience" case. For example, if a disruption event is equally relevant for two groups, such that 50 percent of the disrupted trips are in group "0" and the other 50 percent are in group "1" (without the resilience investment), then we we would also expect to see approximately 50 percent of disrupted trips for group "0" and 50 percent for group "1" if the same disruption event were to occur *with* the resilience investment. If the proportions are significantly different from expected, this may suggest that the benefits of the resilience investment are not equitably distributed. If instead the proportions were 70 percent and 30 percent in the "no resilience" case, then we would similarly expect to see approximately 70 percent of impacted trips in group "0" and 30 percent in group "1" in the resilience case.

The determination of whether the differences from "expected" are "significant" is based on a chi square test using the p-value that the user provides in the equity configuration file (`equity_metric.config` has 0.05 by default). For example, if the p-value is 0.05 (i.e. 5 percent), and the test shows a difference with a p-value less than 0.05, then we can state that there is a 95 percent chance that there is actually a difference and a 5 percent chance that any deviation was observed through random chance. The output below shows the results specific to this analysis.

## Results Specific to this Analysis

In [ ]:
# Conduct chi-square test

# Expected and observed values for trips

# Calculate the proportion of all disrupted trips in the "NO RESILIENCE" case within each equity category.
# Expect the proportion of all disrupted trips in the "RESILIENCE" case in each equity category to be the same,
# assuming that the resilience investment has equitable benefit for each category.
f_exp = summary['trips_disrupt_noresil']/summary['trips_disrupt_noresil'].sum()

# The actual observed proportions of disrupted trips in "RESILIENCE" case are: 
f_obs = summary['trips_disrupt_resil']/summary['trips_disrupt_resil'].sum()

# Chi-square test
teststat,testp = chisquare(f_obs=f_obs, f_exp=f_exp)
testp = round(testp,5)

# p-value is derived from the configuration file. The default is 0.05
# Check whether the p-value that resulted from the chi square test is less than the p-value in the configuration file
if testp < pval:
    print("The proportions of disrupted trips in the 'resilience case' differed significantly from expected\nwith a p-value of {}, suggesting that the benefits of resilience may not be distributed equitably.".format(p))
    print("The chi square statistic (a measure of the difference between the observed and expected proportions)\nwas {} with a p-value of {}, which is less than the user-supplied p-value of {}.".format(teststat,testp,p))
else:
    print("The proportions of disrupted trips in the 'resilience case' did not differ significantly from expected, suggesting that the resilience benefits may be equitably distributed.")
    print("The p-value resulting from the chi square test was {}, which is greater than the user-supplied p-value of {}.".format(testp,pval))

In [ ]:
# Conversion to HTML has moved to TAZ_metrics.py
# !jupyter nbconvert MetricsByTAZ_categorical.ipynb --to html --no-input